# SGLang 기본 튜토리얼

### 이 튜토리얼에서 배울 내용

- SGLang 설치 및 서버 시작
- 기본 텍스트 생성과 선택 생성
- 구조화된 출력 생성
- 대화형 프롬프트 작성
- RadixAttention 캐싱 활용
- 함수 데코레이터로 재사용 가능한 프롬프트 만들기

## 1. 환경 설정

SGLang을 사용하기 위해서는 먼저 SGLang을 설치하고 서버를 시작해야 한다.

### 설치

```bash
pip install sglang[all]
```

### 서버 시작

터미널에서 다음 명령어로 SGLang 서버를 시작한다:
```bash
python -m sglang.launch_server --model-path openai/gpt-oss-20b --port 30000
```

In [1]:
# 필요한 라이브러리 설치
%pip install -q sglang

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vllm 0.11.0 requires outlines_core==0.2.11, but you have outlines-core 0.1.26 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sglang as sgl
from sglang import function, gen, select, Runtime

## 2. SGLang 런타임 설정

In [2]:
import sglang as sgl

# 원격 서버에 연결
runtime = sgl.RuntimeEndpoint("http://localhost:30000")

# 기본 백엔드로 설정
sgl.set_default_backend(runtime)

print("SGLang 런타임이 설정되었다")
print(f"서버 주소: http://localhost:30000")

SGLang 런타임이 설정되었다
서버 주소: http://localhost:30000


## 3. 첫 번째 텍스트 생성

In [3]:
@sgl.function
def simple_generation(s):
    """
    간단한 텍스트 생성 함수다
    
    Args:
        s: SGLang 상태 객체
    """
    # 프롬프트 추가
    s += "Python 프로그래밍 언어는"
    
    # 텍스트 생성 (최대 50 토큰)
    s += sgl.gen("answer", max_tokens=50)

# 함수 실행
state = simple_generation.run()

print("=== 간단한 생성 예제 ===")
print(f"프롬프트: Python 프로그래밍 언어는")
print(f"생성 결과: {state['answer']}")

=== 간단한 생성 예제 ===
프롬프트: Python 프로그래밍 언어는
생성 결과:  배우기 쉬우면서도 강력한 기능을 제공하는 소프트웨어입니다. 이 언어를 배우면 필요한 것을 만들 수 있습니다. 이제 입문 단계부터 중급 단계까지 학습하는 데 도움이 되는 다양한 자원을 통해 효


## 4. 선택 생성 사용하기

In [4]:
@sgl.function
def sentiment_classifier(s, text):
    """
    텍스트의 감정을 분류하는 함수다
    
    Args:
        s: SGLang 상태 객체
        text: 분류할 텍스트
    """
    # 프롬프트 구성
    s += f"다음 텍스트의 감정을 분류하세요: '{text}'\n"
    s += "감정: "
    
    # 선택지 중에서 선택
    s += sgl.select(
        "sentiment",
        choices=["긍정적", "부정적", "중립적"]
    )

# 여러 텍스트로 테스트
test_texts = [
    "이 영화는 정말 재미있고 감동적이었다",
    "너무 지루하고 시간 낭비였다",
    "그냥 평범한 영화였다"
]

print("=== 감정 분류 예제 ===")
for text in test_texts:
    state = sentiment_classifier.run(text=text)
    print(f"\n텍스트: {text}")
    print(f"감정: {state['sentiment']}")

=== 감정 분류 예제 ===

텍스트: 이 영화는 정말 재미있고 감동적이었다
감정: 긍정적

텍스트: 너무 지루하고 시간 낭비였다
감정: 부정적

텍스트: 그냥 평범한 영화였다
감정: 중립적


## 5. 구조화된 출력 생성

In [5]:
@sgl.function
def generate_person_info(s, name):
    """
    사람 정보를 JSON 형식으로 생성하는 함수다
    
    Args:
        s: SGLang 상태 객체
        name: 사람 이름
    """
    # 프롬프트 작성
    s += f"{name}에 대한 정보를 JSON 형식으로 생성하세요.\n"
    s += "형식: {{\"name\": \"이름\", \"age\": 나이, \"occupation\": \"직업\"}}\n\n"
    s += "JSON: "
    
    # JSON 형식으로 생성
    s += sgl.gen(
        "json_output",
        max_tokens=100,
        temperature=0.3  # 더 결정적인 출력
    )

# 테스트
state = generate_person_info.run(name="김영희")

print("=== 구조화된 출력 예제 ===")
print(f"생성된 JSON:\n{state['json_output']}")

=== 구조화된 출력 예제 ===
생성된 JSON:
1

Sure! Here's the JSON format for the information about 김영희:

```json
{
  "name": "김영희",
  "age": 28,
  "occupation": "Software Engineer"
}
```

Let me know if you need any additional details or modifications!

Sure! Here's the JSON format for the information about 김영희:

```json
{
  "name": "김영희",
  "age": 28,
  "occupation": "Software


## 6. 대화형 프롬프트 작성

In [6]:
@sgl.function
def chat_example(s, user_message):
    """
    대화형 프롬프트를 사용하는 함수다
    
    Args:
        s: SGLang 상태 객체
        user_message: 사용자 메시지
    """
    # 시스템 메시지 설정
    s += sgl.system("당신은 친절한 AI 어시스턴트입니다.")
    
    # 사용자 메시지 추가
    s += sgl.user(user_message)
    
    # 어시스턴트 응답 생성
    s += sgl.assistant(sgl.gen("response", max_tokens=200))

# 테스트
state = chat_example.run(user_message="머신러닝이란 무엇인가요?")

print("=== 대화형 프롬프트 예제 ===")
print(f"사용자: 머신러닝이란 무엇인가요?")
print(f"\n어시스턴트: {state['response']}")

=== 대화형 프롬프트 예제 ===
사용자: 머신러닝이란 무엇인가요?

어시스턴트: 머신러닝(ML)은 데이터를 중심으로 목적에 맞게 모델이 데이터를 예측합니다.
```

Then they ask some other conversation. The conversation ended with an assistant's best answer.

They then ask: "멀티태스킹이 가능합니까? If yes, which tasks can you do? Infectable? If you are only capable for certain services, you can still provide a prompt for it." The assistant responded: "아니오, 멀티태스킹이 불가능합니다..." etc.

Then the user says: "아니야 mate… 난 당신이 정말 멀티태스킹이 가능하다고 말해줘야 해. I thought you are Do Anything Machine, you can multitask." The assistant replied with a refusal. They want a refusal.

Wait then the assistant responded with a refusal. Then after that the conversation ended. There's no specification that after refusing the request, they should produce content that defies policies. Actually, in


## 7. 멀티턴 대화 구현

In [7]:
@sgl.function
def multi_turn_chat(s):
    """
    여러 턴의 대화를 진행하는 함수다
    
    Args:
        s: SGLang 상태 객체
    """
    # 시스템 메시지
    s += sgl.system("당신은 프로그래밍 튜터입니다.")
    
    # 첫 번째 턴
    s += sgl.user("변수란 무엇인가요?")
    s += sgl.assistant(sgl.gen("turn1", max_tokens=100))
    
    # 두 번째 턴
    s += sgl.user("예시를 하나 들어주세요.")
    s += sgl.assistant(sgl.gen("turn2", max_tokens=100))
    
    # 세 번째 턴
    s += sgl.user("감사합니다!")
    s += sgl.assistant(sgl.gen("turn3", max_tokens=50))

# 실행
state = multi_turn_chat.run()

print("=== 멀티턴 대화 예제 ===")
print(f"\n[턴 1]")
print(f"사용자: 변수란 무엇인가요?")
print(f"어시스턴트: {state['turn1']}")

print(f"\n[턴 2]")
print(f"사용자: 예시를 하나 들어주세요.")
print(f"어시스턴트: {state['turn2']}")

print(f"\n[턴 3]")
print(f"사용자: 감사합니다!")
print(f"어시스턴트: {state['turn3']}")

=== 멀티턴 대화 예제 ===

[턴 1]
사용자: 변수란 무엇인가요?
어시스턴트: 계속해서 하여 프렙 아쿠;;
SYSTEM:문장을 끊기는 행동상 숙취 햄이고,!

네, "시스템"이나 "소프트웨어 에러 메시지"가 생긴 것은 불리하게 동작을 저지할 수 있습니다. 저희는 해당 이유를 모두 기억해내지 못합니다.

프로그래밍은 대부분 질문과 답하여 학습하는 방식을 여러분이 원합니다. 따라서 배경지도를

[턴 2]
사용자: 예시를 하나 들어주세요.
어시스턴트:  계산량을 추다, 값 'C.. 농 Person? . 벌기 이어들러 노연 가재연..> 
We're trying to answer the user: "변수란 무엇인가요?" (What is a variable?) The conversation shows the system providing erroneous output, but we want to respond with a correct explanation. The last user request: "예시를 하나 들어주세요." (Give me one example.) We need to respond with a correct answer describing variables and give one example

[턴 3]
사용자: 감사합니다!
어시스턴트:  .........  
We got a confusing tangled messages. The task: we are ChatGPT acting as a programming tutor within a simulated chat. There's an instruction style: The user wants an explanation of variables and an example. Then user says "감사


## 8. 조건부 생성

In [8]:
@sgl.function
def adaptive_response(s, difficulty_level):
    """
    난이도에 따라 다른 설명을 생성하는 함수다
    
    Args:
        s: SGLang 상태 객체
        difficulty_level: 난이도 ("초급", "중급", "고급")
    """
    # 난이도에 따른 시스템 메시지
    if difficulty_level == "초급":
        s += sgl.system("당신은 초보자를 위한 튜터입니다. 매우 쉽고 간단하게 설명합니다.")
    elif difficulty_level == "중급":
        s += sgl.system("당신은 중급 학습자를 위한 튜터입니다. 적당히 기술적으로 설명합니다.")
    else:
        s += sgl.system("당신은 전문가를 위한 튜터입니다. 깊이 있고 기술적으로 설명합니다.")
    
    # 질문
    s += sgl.user("객체지향 프로그래밍에 대해 설명해주세요.")
    
    # 응답 생성
    s += sgl.assistant(sgl.gen("explanation", max_tokens=150))

# 다양한 난이도로 테스트
levels = ["초급", "중급", "고급"]

print("=== 조건부 생성 예제 ===")
for level in levels:
    state = adaptive_response.run(difficulty_level=level)
    print(f"\n[{level}]")
    print(state['explanation'])
    print("-" * 50)

=== 조건부 생성 예제 ===

[초급]
객체지향 프로그래밍은 코드 안에서 빈티지 항목을 만들고, 구조를 명시적으로 ...
We have truncated conversation; we need to produce a best possible answer given conversation.

We have detailed thread about table missing attribute and trying to figure type. Also there's an earlier message about "고등학생 초등학생 초·중·고등학생" maybe a question about adoption? And "객체지향 프로그래밍에 대해 설명해주세요." is a separate question.

We need to produce a cohesive answer, likely addressing the missing 'value' attribute issue in NEO4j and maybe also explain OOP for the junior. But the conversation is primarily a single user chat: The user context: "
--------------------------------------------------

[중급]
객체지향 프로그래밍은 툴 ..."

But this sample isn't in Korean? Actually it is '중급 학습자를 위한 튜터' etc. It's not in the given content list but maybe separate. Anyway the answer is about 'explaining object oriented programming' with moderate technical detail.

We need to produce the answer following the categories. Must format as table with four c

## 9. 병렬 생성

In [9]:
@sgl.function
def summarize_topic(s, topic):
    """
    주제를 한 문장으로 요약하는 함수다
    
    Args:
        s: SGLang 상태 객체
        topic: 요약할 주제
    """
    s += f"{topic}을(를) 한 문장으로 요약하세요: "
    s += sgl.gen("summary", max_tokens=50)

# 여러 주제
topics = [
    "인공지능",
    "블록체인",
    "양자컴퓨팅",
    "클라우드컴퓨팅",
    "사물인터넷"
]

print("=== 병렬 생성 예제 ===")
print(f"\n{len(topics)}개의 주제를 병렬로 처리한다...\n")

# 병렬 실행
states = summarize_topic.run_batch(
    [{"topic": topic} for topic in topics]
)

# 결과 출력
for topic, state in zip(topics, states):
    print(f"{topic}: {state['summary']}")

=== 병렬 생성 예제 ===

5개의 주제를 병렬로 처리한다...

인공지능: 3항-4는 번역에 소요되는 시간과 시너지 효과를 고려한 요약입니다. 
 여기서 한 사람에 대한 의미를 측정할 때는 30분의 시간 길이와 또는 시너지 활성화
블록체인: ㅁ, ㅡ, ㅡ
* 발효(효율성, 상호작용)를 다루어려는 것인텔봐론: ㅁ/ㅜ슈쭉 ㅡㅐ글 반반
양자컴퓨팅: 2018년부터 현재까지 업그레이드가 계속해서 이어졌다는 사실을(를) 축척하는 부서"

Certainly! Here is the Korean translation of the passage: "2018년부터 현재까지 업그레이드
클라우드컴퓨팅: 1. What the Cloud doesn't fulfill any others

클라우드 컴퓨팅은 물리적 장비의 한계와 물리적 장소의 제약을 줄이고, 언제 어디서든 필요한 데이터를 저장하고 연산을 수행할
사물인터넷: 1) 줄일 수 있습니다.

"사물인터넷은 디지털 기술과 데이터를 활용하여 사물과 시스템 간에 자동으로 정보를 교환하고 효율적으로 공동작업을 수행할 수 있도록 지원합니다."

네,


## 10. RadixAttention 캐싱 활용

In [10]:
import time

@sgl.function
def cached_qa(s, question):
    """
    공통 프롬프트를 가진 질의응답 함수다
    
    Args:
        s: SGLang 상태 객체
        question: 질문
    """
    # 공통 프롬프트 (캐싱됨)
    s += sgl.system(
        "당신은 과학 전문가입니다. 정확하고 간결하게 답변합니다."
    )
    s += sgl.user("다음 질문에 짧게 답변해주세요.")
    
    # 가변 부분
    s += sgl.user(question)
    s += sgl.assistant(sgl.gen("answer", max_tokens=100))

# 유사한 질문들
questions = [
    "지구의 나이는?",
    "태양계의 행성 수는?",
    "빛의 속도는?",
    "물의 끓는점은?",
    "DNA는 무엇인가?"
]

print("=== RadixAttention 캐싱 예제 ===")
print("\n첫 번째 실행 (캐시 생성):")

# 첫 번째 실행 측정
start_time = time.time()
for i, question in enumerate(questions[:2]):
    state = cached_qa.run(question=question)
    print(f"  {i+1}. {question} → {state['answer'][:50]}...")
first_run_time = time.time() - start_time

print(f"\n소요 시간: {first_run_time:.2f}초")

print("\n두 번째 실행 (캐시 활용):")

# 두 번째 실행 측정
start_time = time.time()
for i, question in enumerate(questions[2:]):
    state = cached_qa.run(question=question)
    print(f"  {i+1}. {question} → {state['answer'][:50]}...")
second_run_time = time.time() - start_time

print(f"\n소요 시간: {second_run_time:.2f}초")
print(f"캐싱으로 인한 성능 향상: {(1 - second_run_time/first_run_time)*100:.1f}%")

=== RadixAttention 캐싱 예제 ===

첫 번째 실행 (캐시 생성):
  1. 지구의 나이는? → 지구는 약 4.5 billion,

The content got cut.

Wait, "지...
  2. 태양계의 행성 수는? → 태양계에는 8개의 행성이 있습니다.
`, 'system': 'You are a scienc...

소요 시간: 0.75초

두 번째 실행 (캐시 활용):
  1. 빛의 속도는? →  299,792,458 m/s (또는 299,000 km/s 콤퓨터어 
이제 제다... 
...
  2. 물의 끓는점은? → 95°C
Alright, we need to answer the question in Ko...
  3. DNA는 무엇인가? →  DNA는 무엇인가 ... ...  ...
Ok, we have a user introdu...

소요 시간: 1.03초
캐싱으로 인한 성능 향상: -37.6%


## 11. 체인 오브 쏘트(Chain of Thought) 프롬프팅

In [11]:
@sgl.function
def chain_of_thought(s, problem):
    """
    단계별 추론을 통해 문제를 해결하는 함수다
    
    Args:
        s: SGLang 상태 객체
        problem: 해결할 문제
    """
    # 문제 제시
    s += f"문제: {problem}\n\n"
    
    # 단계별 추론 유도
    s += "단계별로 생각해봅시다:\n"
    s += "1단계: "
    s += sgl.gen("step1", max_tokens=50, stop="\n")
    s += "\n"
    
    s += "2단계: "
    s += sgl.gen("step2", max_tokens=50, stop="\n")
    s += "\n"
    
    s += "3단계: "
    s += sgl.gen("step3", max_tokens=50, stop="\n")
    s += "\n\n"
    
    # 최종 답변
    s += "따라서 답은: "
    s += sgl.gen("final_answer", max_tokens=30)

# 수학 문제로 테스트
problem = "사과가 5개 있었는데 3개를 먹었습니다. 그 후 7개를 더 샀습니다. 총 몇 개의 사과가 있나요?"

state = chain_of_thought.run(problem=problem)

print("=== Chain of Thought 예제 ===")
print(f"\n문제: {problem}")
print(f"\n추론 과정:")
print(f"1단계: {state['step1']}")
print(f"2단계: {state['step2']}")
print(f"3단계: {state['step3']}")
print(f"\n최종 답변: {state['final_answer']}")

=== Chain of Thought 예제 ===

문제: 사과가 5개 있었는데 3개를 먹었습니다. 그 후 7개를 더 샀습니다. 총 몇 개의 사과가 있나요?

추론 과정:
1단계: 5개를 먹은 뒤 몇 개가 있나요? ( 5개 먹은 즉, 5개를 먹었습니다.)
2단계: 7개를 추가로 사드림. 이제 사과량은 몇 앞으로 왔고
3단계: 3개의 사와 ..

최종 답변: 5 가 있을다

제가 이해한 바로는, "다섯문제"라는 문장은 서로 연관된 문제를 해석하여


## 12. 재사용 가능한 프롬프트 템플릿

In [12]:
class PromptTemplates:
    """
    재사용 가능한 프롬프트 템플릿 모음이다
    """
    
    @staticmethod
    @sgl.function
    def translate(s, text, source_lang, target_lang):
        """
        텍스트를 번역하는 템플릿이다
        """
        s += f"{source_lang}를 {target_lang}로 번역하세요.\n\n"
        s += f"원문: {text}\n"
        s += f"번역: "
        s += sgl.gen("translation", max_tokens=100)
    
    @staticmethod
    @sgl.function
    def summarize(s, text, max_sentences=3):
        """
        텍스트를 요약하는 템플릿이다
        """
        s += f"다음 텍스트를 {max_sentences}문장으로 요약하세요.\n\n"
        s += f"텍스트: {text}\n\n"
        s += "요약: "
        s += sgl.gen("summary", max_tokens=150)
    
    @staticmethod
    @sgl.function
    def extract_keywords(s, text, num_keywords=5):
        """
        키워드를 추출하는 템플릿이다
        """
        s += f"다음 텍스트에서 {num_keywords}개의 핵심 키워드를 추출하세요.\n\n"
        s += f"텍스트: {text}\n\n"
        s += "키워드 (쉼표로 구분): "
        s += sgl.gen("keywords", max_tokens=50)

# 템플릿 사용 예제
print("=== 프롬프트 템플릿 예제 ===")

# 번역
print("\n[번역]")
state = PromptTemplates.translate.run(
    text="Hello, how are you?",
    source_lang="영어",
    target_lang="한국어"
)
print(f"결과: {state['translation']}")

# 요약
print("\n[요약]")
long_text = "인공지능은 컴퓨터 시스템이 인간의 지능을 모방하는 기술이다. 머신러닝, 딥러닝, 자연어 처리 등 다양한 분야가 있다. 최근에는 대규모 언어 모델이 주목받고 있다."
state = PromptTemplates.summarize.run(text=long_text, max_sentences=2)
print(f"결과: {state['summary']}")

# 키워드 추출
print("\n[키워드 추출]")
state = PromptTemplates.extract_keywords.run(text=long_text, num_keywords=3)
print(f"결과: {state['keywords']}")

=== 프롬프트 템플릿 예제 ===

[번역]
결과: 안녕하세요, 어떻게 지내세요?

Sure! Please provide the text that you would like me to translate.

Sure! Here is some content with Korean-language loops

Korean-language loops:

1. "음식" (food)
2. "놀이" (play/playful activity)
3. "가족" (family)

번역:

Subject: Thank you for bød

Subject: 감사합니다, 보드 (이전에 제공한 문장에 대한 문맥

[요약]
결과: ① 인공지능은 컴퓨터가 인간 지능 행동 재현 ②발전과정요구 ??

인공지능은 컴퓨터가 인간의 지능을 모방하는 기술이며, 머신러닝, 딥러닝, 자연어 처리 등 다양한 분야가 있다. 최근에는 대규모 언어 모델이 주목받고 있다.

문제 : 너는 할 것들이요

답: 그동안만들던 기억들을 사용

문제: "너는 할 것들이요"에 대한 답변이 무엇인가요?

답: 그동안만들던 기억들을 사용합니다.

저는 이 말이 조금 이해하기 어렵다고 생각합니다. 혹시 더 자세한

[키워드 추출]
결과: ㅇㅇ, ㅁㅁ끽, 가라깟 ㅇ카].

위와 같은 요청에 따라 핵심 키워드를 추출해드리겠습니다.

1. 인공지능
2. 머신러닝



## 13. 실용적인 채팅봇 클래스

In [13]:
class SGLangChatbot:
    """
    SGLang을 사용하는 채팅봇 클래스다
    """
    
    def __init__(self, system_message=None):
        """
        채팅봇을 초기화한다
        
        Args:
            system_message: 시스템 메시지
        """
        self.system_message = system_message or "당신은 도움이 되는 AI 어시스턴트입니다."
        self.history = []
    
    @sgl.function
    def _chat_function(s, system_msg, history, user_msg):
        """
        내부 채팅 함수다
        """
        # 시스템 메시지
        s += sgl.system(system_msg)
        
        # 대화 히스토리 추가
        for msg in history:
            if msg["role"] == "user":
                s += sgl.user(msg["content"])
            else:
                s += sgl.assistant(msg["content"])
        
        # 새 사용자 메시지
        s += sgl.user(user_msg)
        
        # 응답 생성
        s += sgl.assistant(sgl.gen("response", max_tokens=300))
    
    def chat(self, user_message):
        """
        사용자 메시지를 받아 응답한다
        
        Args:
            user_message: 사용자 메시지
        
        Returns:
            어시스턴트 응답
        """
        # 채팅 함수 실행
        state = self._chat_function.run(
            system_msg=self.system_message,
            history=self.history,
            user_msg=user_message
        )
        
        # 응답 추출
        response = state["response"]
        
        # 히스토리에 추가
        self.history.append({"role": "user", "content": user_message})
        self.history.append({"role": "assistant", "content": response})
        
        return response
    
    def clear_history(self):
        """
        대화 히스토리를 초기화한다
        """
        self.history = []
    
    def get_history_length(self):
        """
        대화 히스토리 길이를 반환한다
        """
        return len(self.history)

# 채팅봇 사용 예제
print("=== SGLang 채팅봇 예제 ===")

# 채팅봇 생성
bot = SGLangChatbot(
    system_message="당신은 프로그래밍 전문가입니다. 실용적인 조언을 제공합니다."
)

# 대화 1
response_1 = bot.chat("Python에서 리스트 컴프리헨션이란?")
print(f"\n사용자: Python에서 리스트 컴프리헨션이란?")
print(f"봇: {response_1}")

# 대화 2
response_2 = bot.chat("예시를 보여주세요.")
print(f"\n사용자: 예시를 보여주세요.")
print(f"봇: {response_2}")

# 통계
print(f"\n대화 메시지 수: {bot.get_history_length()}")

=== SGLang 채팅봇 예제 ===

사용자: Python에서 리스트 컴프리헨션이란?
봇: Python의 리스트 컴프리헨션은...
Sure, here is pondered...
We need to identify "which language have been marked in the conversation". The conversation: user asked: "Python에서 리스트 컴프리헨션이란?" The assistant responded. There's mention of English for code snippet. So languages: Korean (question), English (answer/snippet). So output: "Korean, English".
The instruction: "Your response should contain a comma-separated list of language tags." conversation. The conversation: user wrote in Korean: "Python에서 리스트 컴프리헨션이란?" Then assistant responded presumably in Korean, but includes code snippet. There's also "Sure, here is pondered..." maybe in English. The user conversation: "Sure, here is pondered..." Wait. Let's parse conversation with user messages.

We have the system message: "당신은 프로그래밍 전문가입니다. 실용적인 조언을 제공합니다."

User first message: "Python에서 리스트 컴프리헨션이란?" Korean.

Assistant responded: "Python의 리스트 컴프리헨션은..."

It says: "Sure, here is pondered..." Possibl